# YOLOv8 FSOCO Training with Ultralytics CLI

**Essential steps only:**
1. Setup Ultralytics (YOLOv8)
2. Convert FSOCO labels (Supervisely YOLO)
3. Train
4. Inference

**Run from:** `driverless-ml-dev/perceptions/camera-pipeline/notebooks/`

**Activate venv in root/driverless-ml-dev directory, and connect kernel for this notebook to it**
open vsc terminal via ctrl+shift+`

first time only:
```
cd ~/driverless-ml-dev
python -m venv venv
source venv/bin/activate
pip install ipykernel
python -m ipykernel install --name driverless-ml --display-name "Python (driverless-ml)" --user
```
then type `>developer reload` in the search bar at the top

to activate venv
```
cd /root/driverless-ml-dev
source venv/bin/activate
```
then type `>developer reload` in the search bar at the top

install libraries with: `pip install ultralytics pillow tqdm pyyaml`

In [ ]:
import torch

torch.cuda.is_available()

## Check Ultralytics Installation

Verify that Ultralytics is installed and CUDA is available.

In [ ]:
!yolo checks

In [ ]:
# uncomment below to create and install a virtual environment
# %cd ~/driverless-ml-dev
# !python -m venv venv
# !source venv/bin/activate
# !venv/bin/python -m pip install --upgrade pip ipykernel
# !venv/bin/python -m ipykernel install --name driverless-ml --display-name "Python (driverless-ml)" --user

**Activate venv in root/driverless-ml-dev directory, and connect kernel for this notebook to it**

## 1. Setup

In [ ]:
%pip install ultralytics pillow tqdm pyyaml

In [ ]:

import os
import sys
from pathlib import Path
import mlflow

# Paths
ROOT = Path.cwd().parent.parent.parent  # driverless-ml-dev/
print(ROOT)
DATA_DIR = ROOT / 'ml_data'
FSOCO_RAW = ROOT / 'ml_data/fsoco_raw'  # download fsoco dataset
FSOCO_MOD = ROOT / 'ml_data/fsoco_mod'  # copy of raw for preprocessing
FSOCO_YOLO = ROOT / 'ml_data/fsoco_yolo'  # final dataset

print(f"Root: {ROOT}")
print(f"Data: {DATA_DIR}")
print(f"Fsoco_raw: {FSOCO_RAW}")
print(f"Fsoco_mod: {FSOCO_MOD}")
print(f"Fsoco_yolo: {FSOCO_YOLO}")

## 2. Download FSOCO Dataset

**Manual step required:**
1. Visit: https://fsoco.github.io/fsoco-dataset/download
2. Download Bounding Boxes dataset (24GB)
3. Extract to: `root/ml-data/perceptions/`

Expected structure:
```
root/ml-data/perceptions/
└── fsoco_raw/
    ├── team1/ann/       # JSON annotations
    ├── team1/img/       # Images
    └── meta.json
```

In [ ]:
import os
import shutil
from pathlib import Path

def copy_and_flatten_dataset(source_path: Path, dest_path: Path):
    """
    flattening raw dataset:
    source/
      - meta.json
      - teamA/{ann,img}
      - teamB/{ann,img}
      ...
    ->
    dest/
      - ann/
      - img/
      - meta.json
    """
    if not source_path.exists() or not any(source_path.iterdir()):
        raise FileNotFoundError(f"Source dataset not found or empty: {source_path}")

    if dest_path.exists() and any(dest_path.iterdir()):
        print(f"Skipping copy: destination already populated {dest_path}")
        return

    dest_path.mkdir(parents=True, exist_ok=True)

    for item in dest_path.iterdir():
        if item.is_dir():
            shutil.rmtree(item)
        else:
            item.unlink()

    ann_out = dest_path / "ann"
    img_out = dest_path / "img"
    ann_out.mkdir(exist_ok=True)
    img_out.mkdir(exist_ok=True)

    meta_src = source_path / "meta.json"
    if meta_src.exists():
        shutil.copy2(meta_src, dest_path / "meta.json")

    ann_count = 0
    img_count = 0

    for team_dir in source_path.iterdir():
        print(f"Processing team directory: {team_dir.name}")
        if not team_dir.is_dir():
            print(f"Skipping non-directory item: {team_dir.name}")
            continue

        if team_dir.name in {"ann", "img"}:
            continue

        ann_dir = team_dir / "ann"
        img_dir = team_dir / "img"

        if ann_dir.exists():
            for f in ann_dir.iterdir():
                if f.is_file():
                    dest = ann_out / f"{team_dir.name}_{f.name}"
                    shutil.copy2(f, dest)
                    ann_count += 1

        if img_dir.exists():
            for f in img_dir.iterdir():
                if f.is_file():
                    dest = img_out / f"{team_dir.name}_{f.name}"
                    shutil.copy2(f, dest)
                    img_count += 1

    print(f"Copy+flatten complete {ann_out} ({ann_count} files), {img_out} ({img_count} files)")


raw_exists = FSOCO_RAW.exists()
raw_has_content = os.path.isdir(FSOCO_RAW) if raw_exists else False

# used later on to skip ratio filtering since it assumes you already ran it
mod_exists = FSOCO_MOD.exists()
if raw_exists and raw_has_content:
    print("FSOCO dataset found")
    if FSOCO_MOD.exists() and any(FSOCO_MOD.iterdir()):
        print(f"Skipping flatten: working copy already populated {FSOCO_MOD}")
    else:
        for item in FSOCO_RAW.iterdir():
            print(f"  - {item.name}")
        FSOCO_MOD.mkdir(parents=True, exist_ok=True)
        copy_and_flatten_dataset(FSOCO_RAW, FSOCO_MOD)
else:
    raise FileNotFoundError(f"Download FSOCO dataset to: {FSOCO_RAW}")


In [ ]:
if not FSOCO_MOD.exists():
    raise FileNotFoundError(
        f"Working dataset directory not found: {FSOCO_MOD}. Run the preparation step and ensure the directory exists."
    )

if not any(FSOCO_MOD.iterdir()):
    raise RuntimeError(
        f"Working dataset directory is empty: {FSOCO_MOD}. Run the preparation step to copy data before proceeding."
    )

working_dataset = FSOCO_MOD
print(f"Inspecting dataset at: {working_dataset}")

bb_path = working_dataset / 'ann'
print(bb_path)
print("ann contents:")
if bb_path.exists():
    items = list(os.listdir(bb_path))[:5]
    for item in items:
        print(f"  {item}")
else:
    print("no ann directory found")

img_path = working_dataset / 'img'

print("\nimg contents:")
if img_path.exists():
    items = list(os.listdir(img_path))[:5]
    for item in items:
        print(f"  {item}")
else:
    print("no image directory found")

print("\nmeta.json:")
meta_path = working_dataset / 'meta.json'

if meta_path.exists():
    import json
    meta = json.load(open(meta_path))
    print(json.dumps(meta, indent=2)[:500])
else:
    print("meta.json not found")

## 3. Filter Images by Aspect Ratio

Remove images that don't meet minimum aspect ratio requirements (width/height).

In [ ]:
from PIL import Image
import os
from tqdm import tqdm
from collections import Counter

def filter_images_by_aspect_ratio(fsoco_mod, min_ratio=1.0, max_ratio=3.0):
    img_dir = fsoco_mod / 'images'
    if not img_dir.exists():
        img_dir = fsoco_mod / 'img'
    ann_dir = fsoco_mod / 'bounding_boxes'
    if not ann_dir.exists():
        ann_dir = fsoco_mod / 'ann'
    
    if not img_dir.exists():
        print(f"Image directory not found: {img_dir}")
        return [], 0
    
    images = list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))
    print(f"Total images found: {len(images)}")
    
    filtered_images = []
    removed_images = []
    kept_dimensions = []
    removed_dimensions = []
    kept_aspect_ratios = []
    removed_aspect_ratios = []
    
    for img_path in tqdm(images, desc="Filtering images by aspect ratio"):
        try:
            img = Image.open(img_path)
            w, h = img.size
            aspect_ratio = w / h
            
            if min_ratio <= aspect_ratio <= max_ratio:
                filtered_images.append(img_path)
                kept_dimensions.append((w, h))
                kept_aspect_ratios.append(aspect_ratio)
            else:
                removed_images.append((img_path, aspect_ratio, w, h))
                removed_dimensions.append((w, h))
                removed_aspect_ratios.append(aspect_ratio)
                
                img_path.unlink()
                
                ann_path = ann_dir / f"{img_path.name}.json"
                if not ann_path.exists():
                    ann_path = ann_dir / f"{img_path.stem}.json"
                if ann_path.exists():
                    ann_path.unlink()
                
                print(f"Removed: {img_path.name} (aspect ratio: {aspect_ratio:.2f}, dims: {w}x{h})")
                
        except Exception as e:
            print(f"Error processing {img_path.name}: {e}")
            continue
    #- Print detailed statistics -
    print(f"\n{'='*70}")
    print(f"FILTERING STATISTICS")
    print(f"{'='*70}")
    
    print(f"\nOVERALL SUMMARY:")
    print(f"  Total images processed:  {len(images)}")
    if images:
        print(f"  Images kept:           {len(filtered_images)} ({100*len(filtered_images)/len(images):.1f}%)")
        print(f"  Images filtered out:   {len(removed_images)} ({100*len(removed_images)/len(images):.1f}%)")
    else:
        print("  Images kept:           0")
        print("  Images filtered out:   0")
    print(f"  Aspect ratio range:      {min_ratio} - {max_ratio}")
    
    if kept_aspect_ratios:
        print(f"\nKEPT IMAGES - ASPECT RATIO STATS:")
        print(f"  Min aspect ratio:  {min(kept_aspect_ratios):.2f}")
        print(f"  Max aspect ratio:  {max(kept_aspect_ratios):.2f}")
        print(f"  Avg aspect ratio:  {sum(kept_aspect_ratios)/len(kept_aspect_ratios):.2f}")
    
    if kept_dimensions:
        print(f"\nKEPT IMAGES - DIMENSION STATS:")
        dim_counter = Counter(kept_dimensions)
        top_dims = dim_counter.most_common(10)
        print(f"  Unique dimensions: {len(dim_counter)}")
        print(f"  Top 10 dimensions:")
        for (w, h), count in top_dims:
            ratio = w/h
            print(f"    {w}x{h} (ratio {ratio:.2f}): {count} images")
    
    if removed_aspect_ratios:
        print(f"\nREMOVED IMAGES - ASPECT RATIO STATS:")
        print(f"  Min aspect ratio:  {min(removed_aspect_ratios):.2f}")
        print(f"  Max aspect ratio:  {max(removed_aspect_ratios):.2f}")
        print(f"  Avg aspect ratio:  {sum(removed_aspect_ratios)/len(removed_aspect_ratios):.2f}")
    
    if removed_dimensions:
        print(f"\nREMOVED IMAGES - DIMENSION STATS:")
        dim_counter = Counter(removed_dimensions)
        print(f"  Unique dimensions: {len(dim_counter)}")
        print(f"  All removed dimensions:")
        for (w, h), count in sorted(dim_counter.items(), key=lambda x: x[1], reverse=True):
            ratio = w/h
            print(f"    {w}x{h} (ratio {ratio:.2f}): {count} images")
    
    print(f"\n{'='*70}\n")
    
    return filtered_images, len(removed_images)

MIN_ASPECT_RATIO = 1.25  # minimum width/height ratio
MAX_ASPECT_RATIO = 1.80  # maximum width/height ratio

if not FSOCO_MOD.exists():
    raise FileNotFoundError(
        f"Working dataset directory not found: {FSOCO_MOD}. Create it and rerun the preparation step before filtering."
    )

if not any(FSOCO_MOD.iterdir()):
    raise RuntimeError(
        f"Working dataset directory is empty: {FSOCO_MOD}. Run the preparation step to copy data before filtering."
    )

if not mod_exists:
    print(f"Using working dataset: {FSOCO_MOD}")
    filtered_imgs, removed = filter_images_by_aspect_ratio(
        FSOCO_MOD,
        min_ratio=MIN_ASPECT_RATIO,
        max_ratio=MAX_ASPECT_RATIO
    )
    mod_exists = True
    print("Aspect ratio filtering complete. Flag updated to skip on next run.")
else:
    print("Skipping aspect ratio filtering: already applied previously.")

## 4. Convert Labels: Supervisely YOLO

YOLO format: `<class_id> <x_center> <y_center> <width> <height>` (all normalized 0-1)

In [ ]:
import json
import shutil
from PIL import Image
from tqdm import tqdm
import random

def convert_supervisely_to_yolo(fsoco_raw, fsoco_yolo, split=(0.8, 0.1, 0.1)):
    for s in ['train', 'val', 'test']:
        (fsoco_yolo / 'images' / s).mkdir(parents=True, exist_ok=True)
        (fsoco_yolo / 'labels' / s).mkdir(parents=True, exist_ok=True)
    
    # hard code class names
    classes = ['blue_cone', 'unknown_cone', 'orange_cone', 'large_orange_cone', 'yellow_cone']
    class_map = {name: i for i, name in enumerate(classes)}
    print(f"Classes (bounding boxes only): {classes}\n")
    print(f"Class mapping: {class_map}\n")

    ann_dir = fsoco_raw / 'ann'
    img_dir = fsoco_raw / 'img'
    
    ann_files = list(ann_dir.glob('*.json'))
    print(f"Total annotations: {len(ann_files)}")
    
    random.shuffle(ann_files)
    n1 = int(len(ann_files) * split[0])
    n2 = int(len(ann_files) * (split[0] + split[1]))
    splits = {
        'train': ann_files[:n1],
        'val': ann_files[n1:n2],
        'test': ann_files[n2:]
    }
    
    for split_name, anns in splits.items():
        print(f"\nConverting {split_name}: {len(anns)} images")
        
        for ann_path in tqdm(anns):
            img_name = ann_path.stem
            img_path = img_dir / img_name
            
            if not img_path.exists():
                alt_name = img_name.replace('.jpg', '.png') if '.jpg' in img_name else img_name.replace('.png', '.jpg')
                img_path = img_dir / alt_name
                if not img_path.exists():
                    continue

            img = Image.open(img_path)
            w, h = img.size

            ann = json.load(open(ann_path))

            yolo_labels = []
            for obj in ann.get('objects', []):
                cls = obj['classTitle']
                if cls not in class_map or obj['geometryType'] != 'rectangle':
                    continue
                
                points = obj['points']['exterior']
                x1, y1 = points[0]
                x2, y2 = points[1]

                x_center = ((x1 + x2) / 2) / w
                y_center = ((y1 + y2) / 2) / h
                width = abs(x2 - x1) / w
                height = abs(y2 - y1) / h

                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                width = max(0, min(1, width))
                height = max(0, min(1, height))
                
                yolo_labels.append(f"{class_map[cls]} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            if yolo_labels:
                shutil.copy(img_path, fsoco_yolo / 'images' / split_name / img_path.name)
                with open(fsoco_yolo / 'labels' / split_name / f"{img_path.stem}.txt", 'w') as f:
                    f.write('\n'.join(yolo_labels))
    
    print("\nConversion complete")
    print(f"\nDataset splits:")
    for split_name in ['train', 'val', 'test']:
        n_imgs = len(list((fsoco_yolo / 'images' / split_name).glob('*.[jp][pn]g')))
        print(f"  {split_name}: {n_imgs} images")
    
    return classes

classes = None
if (FSOCO_MOD / 'meta.json').exists():
    try:
        meta = json.load(open(FSOCO_MOD / 'meta.json'))
        classes = [c['title'] for c in meta['classes'] if c['shape'] == 'rectangle']
        print(f"Loaded {len(classes)} classes from meta.json: {classes}")
    except Exception as e:
        print(f"Warning: Failed to load classes from meta.json: {e}")

yolo_exists = FSOCO_YOLO.exists() and any(FSOCO_YOLO.iterdir())

if not (FSOCO_MOD.exists() and any(FSOCO_MOD.iterdir())):
    raise FileNotFoundError(
        f"Working dataset not found or empty: {FSOCO_MOD}\n"
        f"Please run the dataset preparation step first (cell 10)."
    )

if yolo_exists:
    print(f"Skipping conversion: YOLO dataset already populated {FSOCO_YOLO}")
else:
    print(f"Using working dataset: {FSOCO_MOD}")
    classes = convert_supervisely_to_yolo(FSOCO_MOD, FSOCO_YOLO)

if classes is None:
    raise RuntimeError(
        "Failed to load class definitions.\n"
        f"Ensure meta.json exists in {FSOCO_MOD} or run the conversion."
    )

print(f"\nFinal classes list ({len(classes)} classes): {classes}")

## 5. Create Dataset Config (YAML)

In [ ]:
import yaml

if classes is None:
    raise RuntimeError(
        "Classes not defined. Please run the label conversion step first."
    )

config_path = FSOCO_YOLO / 'fsoco.yaml'

if yolo_exists and config_path.exists():
    print(f"Config already exists: {config_path}")
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print(yaml.dump(config, default_flow_style=False))
else:
    config = {
        'path': str(FSOCO_YOLO.absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(classes),
        'names': classes
    }
    
    with open(config_path, 'w') as f:
        yaml.dump(config, f)
    
    print("Config saved:", config_path)
    print(yaml.dump(config, default_flow_style=False))

## 6. YOLOv8 Training Configuration

In [ ]:
import torch
MODEL = 'yolov8s.pt'  # Options: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x; change number to 10, 11
EPOCHS = 100
BATCH = -1  # auto batch size
IMG_SIZE = 640
DEVICE = "0" if torch.cuda.is_available() else "cpu"
PROJECT = 'runs/detect'
NAME = 'fsoco_train'
#add early stopping

print(f"Model: {MODEL}, Epochs: {EPOCHS}, Batch: {BATCH}, Image size: {IMG_SIZE}, Device: {DEVICE}")

In [1]:
!yolo settings tensorboard=True

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 3, in <module>
    from ultralytics.cfg import entrypoint
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/__init__.py", line 12, in <module>
    from ultralytics.utils import ASSETS, SETTINGS
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/__init__.py", line 25, in <module>
    import torch
  File "/usr/local/lib/python3.12/dist-packages/torch/__init__.py", line 2073, in <module>
    _C._initExtension(_manager_path())
  File "/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py", line 1458, in <module>
    from .memory import *  # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/cuda/memory.py", line 599, in <module>
    @deprecated(
     ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/typing_extensions.py", line 2949, in __call__
    import asyncio.coroutines
  File "/usr/lib/python3.12/asyncio/__init__.py", line 8, in <mod

## 6.1 Train YOLOv8 Model

Using the Ultralytics CLI to train the model.



In [ ]:
!yolo detect train data={config_path} model={MODEL} epochs={EPOCHS} imgsz={IMG_SIZE} batch={BATCH} device={DEVICE} project={PROJECT} name={NAME}

## 7. View Results

In [ ]:
from IPython.display import Image as IPImage
from pathlib import Path

runs_dir = Path('runs/detect')
if runs_dir.exists():
    runs = sorted(runs_dir.glob('fsoco_train*'), key=lambda x: x.stat().st_mtime)
    if runs:
        latest = runs[-1]
        print(f"Training run: {latest.name}\n")

        results_img = latest / 'results.png'
        if results_img.exists():
            display(IPImage(filename=str(results_img)))
        
        print(f"\nWeights: {latest / 'weights' / 'best.pt'}")

        with mlflow.start_run(run_name=latest.name):
            mlflow.log_params({
                "model": MODEL,
                "epochs": EPOCHS,
                "batch_size": BATCH,
                "img_size": IMG_SIZE,
                "device": DEVICE
            })
            
            results_csv = latest / 'results.csv'
            if results_csv.exists():
                import pandas as pd
                df = pd.read_csv(results_csv)
                if not df.empty:
                    last_row = df.iloc[-1]
                    for col in df.columns:
                        if col.strip() and col.strip() != 'epoch':
                            try:
                                mlflow.log_metric(col.strip(), float(last_row[col]))
                            except:
                                pass
        
        mlflow.end_run()
    else:
        print("No training runs found")
else:
    print("No runs directory found. Train the model first.")

In [ ]:
# export to tensorrt
from pathlib import Path

runs_dir = Path('runs/detect')
if runs_dir.exists():
    runs = sorted(runs_dir.glob('fsoco_train*'), key=lambda x: x.stat().st_mtime)
    if runs:
        weight = runs[-1] / 'weights' / 'best.pt'
        print(f"Exporting weights: {weight}")
        !yolo export model={weight} format=engine device={DEVICE}
    else:
        print("No training runs found. Train the model first.")
else:
    print("No runs directory found. Train the model first.")

## 7.1 Export

Export the trained model to formats:
- `onnx`: ONNX format 
- `engine`: TensorRT (for jetson)

In [ ]:
#exports
from pathlib import Path

runs_dir = Path('runs/detect')
if runs_dir.exists():
    runs = sorted(runs_dir.glob('fsoco_train*'), key=lambda x: x.stat().st_mtime)
    if runs:
        weight = runs[-1] / 'weights' / 'best.pt'
        print(f"Exporting weights: {weight}\n")

        formats = [
            'onnx',      # ONNX format
            # 'engine',  # TensorRT format
        ]
        
        for fmt in formats:
            print(f"\nExporting to {fmt}...")
            !yolo export model={weight} format={fmt} imgsz={IMG_SIZE}
        
        print("\nExport complete!")
    else:
        print("No training runs found. Train the model first.")
else:
    print("No runs directory found. Train the model first.")

## 8. Run Inference

In [ ]:
from pathlib import Path

runs_dir = Path('runs/detect')
if runs_dir.exists():
    runs = sorted(runs_dir.glob('fsoco_train*'), key=lambda x: x.stat().st_mtime)
    if runs:
        weights = runs[-1] / 'weights' / 'best.pt'
        test_imgs = FSOCO_YOLO / 'images' / 'test'
        
        if weights.exists() and test_imgs.exists():
            print("Running inference with YOLOv8...\n")
            #inference
            !yolo detect predict model={weights} source={test_imgs} imgsz={IMG_SIZE} conf=0.25 save=True name=fsoco_inference project=runs/detect

            # results
            detect_runs = sorted(Path('runs/detect').glob('fsoco_inference*'), 
                               key=lambda x: x.stat().st_mtime)
            if detect_runs:
                print(f"\nResults saved to: {detect_runs[-1]}")

                results = list(detect_runs[-1].glob('*.jpg'))[:3]  # Show first 3 results
                for r in results:
                    print(f"\nShowing: {r.name}")
                    display(IPImage(filename=str(r), width=800))
        else:
            print("Missing weights or test images")
    else:
        print("No training runs found. Train model first.")
else:
    print("No runs directory found. Train model first.")

## 9. Validate Model Performance

In [ ]:
from pathlib import Path

runs_dir = Path('runs/detect')
if runs_dir.exists():
    runs = sorted(runs_dir.glob('fsoco_train*'), key=lambda x: x.stat().st_mtime)
    if runs:
        weights = runs[-1] / 'weights' / 'best.pt'
        
        if weights.exists():
            print("Running validation...\n")
            
            # validation
            !yolo detect val model={weights} data={config_path} imgsz={IMG_SIZE} batch={BATCH} device={DEVICE} save_json=True split=val
            
            print("\nValidation complete!")
        else:
            print("Weights not found")
    else:
        print("No training runs found. Train model first.")
else:
    print("No runs directory found. Train model first.")

In [ ]:
# benchmarking
from pathlib import Path
import pandas as pd
import torch

runs_dir = Path('runs/detect')
if runs_dir.exists():
    candidates = sorted(runs_dir.glob("fsoco_train*/weights/best.pt"), key=lambda p: p.stat().st_mtime)
    
    if candidates:
        W = candidates[-1]
        DATA_YAML = config_path
        IMG = IMG_SIZE
        DEVICE = "0" if torch.cuda.is_available() else "cpu"
        
        print(f"Benchmarking weights: {W}")
        print(f"Data: {DATA_YAML}")
        print(f"Image size: {IMG}")
        print(f"Device: {DEVICE}\n")
        
        # benchmark
        !yolo detect val model={W} data={DATA_YAML} imgsz={IMG} batch=1 device={DEVICE} save_json=True
        
        # speed test
        print("\n" + "="*70)
        print("Running speed benchmark...")
        print("="*70 + "\n")
        
        # speed
        !yolo benchmark model={W} data={DATA_YAML} imgsz={IMG} device={DEVICE}
        
    else:
        print("No trained weights found. Train the model first.")
else:
    print("No runs directory found. Train the model first.")